In [11]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()

bucket = sagemaker.Session().default_bucket()
prefix = "DEMO-breast-cancer-prediction-xgboost-lowlevel"

In [12]:
import pandas as pd
import numpy as np

s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file(
    f"sagemaker-example-files-prod-{boto3.session.Session().region_name}",
    "datasets/tabular/breast_cancer/wdbc.csv",
    filename,
)
data = pd.read_csv(filename, header=None)

# specify columns extracted from wbdc.names
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# save the data
data.to_csv("data.csv", sep=",", index=False)

data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
568,92751,B,7.760,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.000000,...,9.456,30.37,59.16,268.6,0.08996,0.06444,0.00000,0.00000,0.2871,0.07039
76,8610629,B,13.530,10.94,87.91,559.2,0.12910,0.10470,0.06877,0.065560,...,14.080,12.49,91.36,605.5,0.14510,0.13790,0.08539,0.07407,0.2710,0.07191
246,884448,B,13.200,17.43,84.13,541.6,0.07215,0.04524,0.04336,0.011050,...,13.940,27.82,88.28,602.0,0.11010,0.15080,0.22980,0.04970,0.2767,0.07198
52,857374,B,11.940,18.24,75.71,437.6,0.08261,0.04751,0.01972,0.013490,...,13.100,21.33,83.67,527.2,0.11440,0.08906,0.09203,0.06296,0.2785,0.07408
394,903554,B,12.100,17.72,78.07,446.2,0.10290,0.09758,0.04783,0.033260,...,13.560,25.80,88.33,559.5,0.14320,0.17730,0.16030,0.06266,0.3049,0.07081
125,86561,B,13.850,17.21,88.44,588.7,0.08785,0.06136,0.01420,0.011410,...,15.490,23.58,100.30,725.9,0.11570,0.13500,0.08115,0.05104,0.2364,0.07182
61,858981,B,8.598,20.98,54.66,221.8,0.12430,0.08963,0.03000,0.009259,...,9.565,27.04,62.06,273.9,0.16390,0.16980,0.09001,0.02778,0.2972,0.07712
408,90524101,M,17.990,20.66,117.80,991.7,0.10360,0.13040,0.12010,0.088240,...,21.080,25.41,138.10,1349.0,0.14820,0.37350,0.33010,0.19740,0.3060,0.08503


In [14]:
data.shape

(569, 32)

In [15]:
data["diagnosis"] = data["diagnosis"].apply(lambda x: ((x == "M")) + 0)
data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
219,88119002,1,19.530,32.47,128.00,1223.0,0.08420,0.11300,0.11450,0.06637,...,27.90,45.41,180.20,2477.0,0.14080,0.4097,0.39950,0.16250,0.2713,0.07568
393,903516,1,21.610,22.28,144.40,1407.0,0.11670,0.20870,0.28100,0.15620,...,26.23,28.74,172.00,2081.0,0.15020,0.5717,0.70530,0.24220,0.3828,0.10070
432,908194,1,20.180,19.54,133.80,1250.0,0.11330,0.14890,0.21330,0.12590,...,22.03,25.07,146.00,1479.0,0.16650,0.2942,0.53080,0.21730,0.3032,0.08075
440,909411,0,10.970,17.20,71.73,371.5,0.08915,0.11130,0.09457,0.03613,...,12.36,26.87,90.14,476.4,0.13910,0.4082,0.47790,0.15550,0.2540,0.09532
407,905190,0,12.850,21.37,82.63,514.5,0.07551,0.08316,0.06126,0.01867,...,14.40,27.01,91.63,645.8,0.09402,0.1936,0.18380,0.05601,0.2488,0.08151
36,854268,1,14.250,21.72,93.63,633.0,0.09823,0.10980,0.13190,0.05598,...,15.89,30.36,116.20,799.6,0.14460,0.4238,0.51860,0.14470,0.3591,0.10140
310,893783,0,11.700,19.11,74.33,418.7,0.08814,0.05253,0.01583,0.01148,...,12.61,26.55,80.92,483.1,0.12230,0.1087,0.07915,0.05741,0.3487,0.06958
504,915186,0,9.268,12.87,61.49,248.7,0.16340,0.22390,0.09730,0.05252,...,10.28,16.38,69.05,300.2,0.19020,0.3441,0.20990,0.10250,0.3038,0.12520


In [16]:
# data split in three sets, training, validation and batch inference
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

data_train = data[train_list].drop(["id"], axis=1)
data_val = data[val_list].drop(["id"], axis=1)
data_batch = data[batch_list].drop(["diagnosis"], axis=1)
data_batch_noID = data_batch.drop(["id"], axis=1)

In [25]:
data_batch 

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
5,843786,12.450,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,...,15.470,23.75,103.40,741.6,0.1791,0.52490,0.53550,0.17410,0.3985,0.12440
8,844981,13.000,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,...,15.490,30.73,106.20,739.3,0.1703,0.54010,0.53900,0.20600,0.4378,0.10720
16,848406,14.680,20.13,94.74,684.5,0.09867,0.07200,0.07395,0.05259,0.1586,...,19.070,30.88,123.40,1138.0,0.1464,0.18710,0.29140,0.16090,0.3029,0.08216
17,84862001,16.130,20.68,108.10,798.8,0.11700,0.20220,0.17220,0.10280,0.2164,...,20.960,31.48,136.80,1315.0,0.1789,0.42330,0.47840,0.20730,0.3706,0.11420
31,853612,11.840,18.70,77.93,440.6,0.11090,0.15160,0.12180,0.05182,0.2301,...,16.820,28.12,119.40,888.7,0.1637,0.57750,0.69560,0.15460,0.4761,0.14020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,918465,12.070,13.44,77.83,445.2,0.11000,0.09009,0.03781,0.02798,0.1657,...,13.450,15.77,86.92,549.9,0.1521,0.16320,0.16220,0.07393,0.2781,0.08052
530,91858,11.750,17.56,75.89,422.9,0.10730,0.09713,0.05282,0.04440,0.1598,...,13.500,27.98,88.52,552.3,0.1349,0.18540,0.13660,0.10100,0.2478,0.07757
531,91903901,11.670,20.02,75.21,416.2,0.10160,0.09453,0.04200,0.02157,0.1859,...,13.350,28.81,87.00,550.6,0.1550,0.29640,0.27580,0.08120,0.3206,0.08950
549,923465,10.820,24.21,68.89,361.6,0.08192,0.06602,0.01548,0.00816,0.1976,...,13.030,31.45,83.90,505.6,0.1204,0.16330,0.06194,0.03264,0.3059,0.07626


In [17]:
s3_resource = boto3.Session().resource("s3")

train_file = "train_data.csv"
data_train.to_csv(train_file, index=False, header=False)
with open(train_file, "rb") as data:
    s3_resource.Bucket(bucket).upload_fileobj(data, os.path.join(prefix, "train", train_file))

validation_file = "validation_data.csv"
data_val.to_csv(validation_file, index=False, header=False)
with open(validation_file, "rb") as data:
    s3_resource.Bucket(bucket).upload_fileobj(
        data, os.path.join(prefix, "validation", validation_file)
    )

batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)
with open(batch_file, "rb") as data:
    s3_resource.Bucket(bucket).upload_fileobj(data, os.path.join(prefix, "batch", batch_file))

batch_file_noID = "batch_data_noID.csv"
data_batch_noID.to_csv(batch_file_noID, index=False, header=False)
with open(batch_file_noID, "rb") as data:
    s3_resource.Bucket(bucket).upload_fileobj(data, os.path.join(prefix, "batch", batch_file_noID))

In [26]:
%%time
from time import gmtime, strftime
from sagemaker.image_uris import retrieve

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Training job", job_name)

image = retrieve(framework="xgboost", region=boto3.Session().region_name, version="latest")

output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
print("Training artifacts will be uploaded to: {}".format(output_location))

create_training_params = {
    "AlgorithmSpecification": {"TrainingImage": image, "TrainingInputMode": "File"},
    "RoleArn": role,
    "OutputDataConfig": {"S3OutputPath": output_location},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": "ml.m5.4xlarge", "VolumeSizeInGB": 50},
    "TrainingJobName": job_name,
    "HyperParameters": {
        "objective": "binary:logistic",
        "max_depth": "5",
        "eta": "0.2",
        "gamma": "4",
        "min_child_weight": "6",
        "subsample": "0.8",
        "silent": "0",
        "num_round": "100",
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 60 * 60},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
            "ContentType": "text/csv",
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/validation".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
            "ContentType": "text/csv",
        },
    ],
}

sagemaker = boto3.client("sagemaker")
sagemaker.create_training_job(**create_training_params)
status = sagemaker.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]
print(status)

try:
    sagemaker.get_waiter("training_job_completed_or_stopped").wait(TrainingJobName=job_name)
finally:
    status = sagemaker.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]
    print("Training job ended with status: " + status)
    if status == "Failed":
        message = sagemaker.describe_training_job(TrainingJobName=job_name)["FailureReason"]
        print("Training failed with the following error: {}".format(message))
        raise Exception("Training job failed")

Training job xgb-2024-05-29-21-21-07
Training artifacts will be uploaded to: s3://sagemaker-us-west-2-339713066436/DEMO-breast-cancer-prediction-xgboost-lowlevel/output/xgb-2024-05-29-21-21-07
InProgress
Training job ended with status: Completed
CPU times: user 122 ms, sys: 15.6 ms, total: 137 ms
Wall time: 2min


In [27]:
%%time

model_name = job_name
print(model_name)

info = sagemaker.describe_training_job(TrainingJobName=job_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": image, "ModelDataUrl": model_data}

create_model_response = sagemaker.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

xgb-2024-05-29-21-21-07
arn:aws:sagemaker:us-west-2:339713066436:model/xgb-2024-05-29-21-21-07
CPU times: user 27.2 ms, sys: 0 ns, total: 27.2 ms
Wall time: 830 ms


In [28]:
%%time

from time import gmtime, strftime

batch_job_name = "Batch-Transform-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file_noID
)  # use input data without ID column
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, batch_job_name)

request = {
    "TransformJobName": batch_job_name,
    "ModelName": job_name,
    "TransformOutput": {
        "S3OutputPath": output_location,
        "Accept": "text/csv",
        "AssembleWith": "Line",
    },
    "TransformInput": {
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": input_location}},
        "ContentType": "text/csv",
        "SplitType": "Line",
        "CompressionType": "None",
    },
    "TransformResources": {"InstanceType": "ml.m4.xlarge", "InstanceCount": 1},
}

sagemaker.create_transform_job(**request)
print("Created Transform job with name: ", batch_job_name)

# Wait until the job finishes
try:
    sagemaker.get_waiter("transform_job_completed_or_stopped").wait(TransformJobName=batch_job_name)
finally:
    response = sagemaker.describe_transform_job(TransformJobName=batch_job_name)
    status = response["TransformJobStatus"]
    print("Transform job ended with status: " + status)
    if status == "Failed":
        message = response["FailureReason"]
        print("Transform failed with the following error: {}".format(message))
        raise Exception("Transform job failed")

Created Transform job with name:  Batch-Transform-2024-05-29-21-30-39
Transform job ended with status: Completed
CPU times: user 156 ms, sys: 15.8 ms, total: 172 ms
Wall time: 7min 1s


In [29]:
import re


def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [32]:
output_df = get_csv_output_from_s3(output_location, batch_file_noID)
output_df

,0
0,0.467408
1,0.916059
2,0.987475
3,0.991606
4,0.967689
...,...
62,0.009711
63,0.011638
64,0.031410
65,0.058175


In [33]:
%%time

batch_job_name = "Batch-Transform-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
)  # use input data with ID column cause InputFilter will filter it out
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, batch_job_name)

request["TransformJobName"] = batch_job_name
request["TransformInput"]["DataSource"]["S3DataSource"]["S3Uri"] = input_location
request["TransformOutput"]["S3OutputPath"] = output_location

request["DataProcessing"] = {
    "InputFilter": "$[1:]",  # exclude the ID column (index 0)
    "JoinSource": "Input",  # join the input with the inference results
}

sagemaker.create_transform_job(**request)
print("Created Transform job with name: ", batch_job_name)

# Wait until the job finishes
try:
    sagemaker.get_waiter("transform_job_completed_or_stopped").wait(TransformJobName=batch_job_name)
finally:
    response = sagemaker.describe_transform_job(TransformJobName=batch_job_name)
    status = response["TransformJobStatus"]
    print("Transform job ended with status: " + status)
    if status == "Failed":
        message = response["FailureReason"]
        print("Transform failed with the following error: {}".format(message))
        raise Exception("Transform job failed")

Created Transform job with name:  Batch-Transform-2024-05-29-21-52-36
Transform job ended with status: Completed
CPU times: user 167 ms, sys: 9.9 ms, total: 177 ms
Wall time: 7min 1s


In [34]:
output_df = get_csv_output_from_s3(output_location, batch_file)
output_df

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,843786,12.450,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,...,23.75,103.40,741.6,0.1791,0.52490,0.53550,0.17410,0.3985,0.12440,0.467408
1,844981,13.000,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,...,30.73,106.20,739.3,0.1703,0.54010,0.53900,0.20600,0.4378,0.10720,0.916059
2,848406,14.680,20.13,94.74,684.5,0.09867,0.07200,0.07395,0.05259,0.1586,...,30.88,123.40,1138.0,0.1464,0.18710,0.29140,0.16090,0.3029,0.08216,0.987475
3,84862001,16.130,20.68,108.10,798.8,0.11700,0.20220,0.17220,0.10280,0.2164,...,31.48,136.80,1315.0,0.1789,0.42330,0.47840,0.20730,0.3706,0.11420,0.991606
4,853612,11.840,18.70,77.93,440.6,0.11090,0.15160,0.12180,0.05182,0.2301,...,28.12,119.40,888.7,0.1637,0.57750,0.69560,0.15460,0.4761,0.14020,0.967689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,918465,12.070,13.44,77.83,445.2,0.11000,0.09009,0.03781,0.02798,0.1657,...,15.77,86.92,549.9,0.1521,0.16320,0.16220,0.07393,0.2781,0.08052,0.009711
63,91858,11.750,17.56,75.89,422.9,0.10730,0.09713,0.05282,0.04440,0.1598,...,27.98,88.52,552.3,0.1349,0.18540,0.13660,0.10100,0.2478,0.07757,0.011638
64,91903901,11.670,20.02,75.21,416.2,0.10160,0.09453,0.04200,0.02157,0.1859,...,28.81,87.00,550.6,0.1550,0.29640,0.27580,0.08120,0.3206,0.08950,0.031410
65,923465,10.820,24.21,68.89,361.6,0.08192,0.06602,0.01548,0.00816,0.1976,...,31.45,83.90,505.6,0.1204,0.16330,0.06194,0.03264,0.3059,0.07626,0.058175
